In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.insert(0, "..")

from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np

from peekingduck.pipeline.nodes.draw.utils.bbox import draw_bboxes
from peekingduck.pipeline.nodes.draw.utils.general import (
    get_image_size,
    project_points_onto_original_image,
)

from src.custom_nodes.draw.annotation import draw_annotations


In [ ]:
def get_frame(frame_number):
    return (
        Path.cwd().parent
        / "data"
        / "input"
        / "video_1"
        / "bus_boarding_alighting_frames"
        / f"video_1_{frame_number}.jpg"
    )


def preview_frame(frame_path):
    frame = cv2.imread(str(frame_path))

    _, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(frame[:, :, ::-1])


def preview_image(image):
    _, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(image[:, :, ::-1])


In [ ]:
img_path_3441 = get_frame(3441)
img_path_3445 = get_frame(3445)
# img_path_3371 = get_frame(3371)


In [ ]:
bboxes_3441 = np.array([[0.61, 0.31, 0.67, 0.6], [0.66, 0.32, 0.72, 0.59]])
bbox_labels_3441 = np.array(["person", "person"])

img_3441 = cv2.imread(str(img_path_3441))
# draw_bboxes(img_3441, bboxes_3441, bbox_labels_3441, True)
# preview_image(img_3441)


In [ ]:
from sort import Sort

In [ ]:
mot_tracker = Sort("csrt")

In [ ]:
track_ids = mot_tracker.update_and_get_tracks(img_3441, bboxes_3441)
# track_ids = list(map(str, reversed(track_ids)))

In [ ]:
annotations = [{"track_id": id, "idx": str(i)} for i, id in enumerate(track_ids)]
img_3441_copy = img_3441.copy()
draw_bboxes(img_3441_copy, bboxes_3441, bbox_labels_3441, True)
draw_annotations(
    img_3441_copy, bboxes_3441, annotations, (255, 0, 0), (0.2, 0.2), "track_id"
)
draw_annotations(
    img_3441_copy, bboxes_3441, annotations, (255, 0, 0), (0.8, 0.8), "idx"
)
preview_image(img_3441_copy)


In [ ]:
bboxes_3445 = np.array(
    [[0.62, 0.32, 0.67, 0.59], [0.58, 0.34, 0.64, 0.59], [0.66, 0.34, 0.71, 0.59]]
)
bbox_labels_3445 = np.array(["person", "person", "person"])

img_3445 = cv2.imread(str(img_path_3445))
# draw_bboxes(img_3445, bboxes_3445, bbox_labels_3445, True)
# annotations = [{"idx": str(i)} for i, _ in enumerate(bboxes_3445)]
# draw_annotations(
#     img_3445, bboxes_3445, annotations, (255, 0, 0), (0.8, 0.8), "idx"
# )
# preview_image(img_3445)


In [ ]:
track_ids = mot_tracker.update_and_get_tracks(img_3445, bboxes_3445)
# track_ids = list(map(str, rev'ersed(track_ids)))

In [ ]:
annotations = [{"track_id": id, "idx": str(i)} for i, id in enumerate(track_ids)]
img_3445_copy = img_3445.copy()
draw_bboxes(img_3445_copy, bboxes_3445, bbox_labels_3445, True)
draw_annotations(
    img_3445_copy, bboxes_3445, annotations, (255, 0, 0), (0.2, 0.2), "track_id"
)
draw_annotations(
    img_3445_copy, bboxes_3445, annotations, (255, 0, 0), (0.8, 0.8), "idx"
)
preview_image(img_3445_copy)


In [ ]:
## Old code

# bboxes_3365 = np.array([[0.64, 0.28, 0.69, 0.5], [0.72, 0.31, 0.77, 0.55]])
# bbox_labels_3365 = np.array(["person", "person"])

# img_3365 = cv2.imread(str(img_path_3365))
# draw_bboxes(img_3365, bboxes_3365, bbox_labels_3365, True)
# preview_image(img_3365)

next_obj_id = 0
tracked_dict = {}

image_size_3441 = get_image_size(img_3441)
bboxes_xywh_3441 = Utils.xyxyn2xywh(bboxes_3441, *image_size_3441)

for bbox in bboxes_xywh_3441:
    tracker = cv2.TrackerCSRT_create()
    tracker.init(img_3441, tuple(bbox))
    tracked_dict[next_obj_id] = {"tracker": tracker, "bbox": bbox}
    next_obj_id += 1
track_ids = list(map(str, tracked_dict.keys()))

def iou(bbox, candidates):
    bbox_tl, bbox_br = bbox[:2], bbox[:2] + bbox[2:]
    candidates_tl = candidates[:, :2]
    candidates_br = candidates[:, :2] + candidates[:, 2:]

    top_left = np.c_[
        np.maximum(bbox_tl[0], candidates_tl[:, 0])[:, np.newaxis],
        np.maximum(bbox_tl[1], candidates_tl[:, 1])[:, np.newaxis],
    ]
    bottom_right = np.c_[
        np.minimum(bbox_br[0], candidates_br[:, 0])[:, np.newaxis],
        np.minimum(bbox_br[1], candidates_br[:, 1])[:, np.newaxis],
    ]
    width_height = np.maximum(0.0, bottom_right - top_left)

    area_intersection = width_height.prod(axis=1)
    area_bbox = bbox[2:].prod()
    area_candidates = candidates[:, 2:].prod(axis=1)
    return area_intersection / (area_bbox + area_candidates - area_intersection)


image_size_3445 = get_image_size(img_3445)
bboxes_xywh_3445 = xyxyn2xywh(bboxes_3445, *image_size_3445)

obj_tags = [""] * len(bboxes_xywh_3445)
prev_tracked_bbox = [np.array(value["bbox"]) for _, value in tracked_dict.items()]
matching_dict = {}

for bbox in bboxes_xywh_3445:
    ious = iou(np.array(bbox), np.array(prev_tracked_bbox))
    matching_dict[tuple(bbox)] = ious.argmax() if max(ious) >= 0.1 else None

track_ids = []
for key, value in matching_dict.items():
    if value is not None:
        track_ids.append(str(list(tracked_dict)[value]))
    else:
        tracker = cv2.TrackerCSRT_create()
        tracker.init(img_3445, tuple(bbox))
        tracked_dict[next_obj_id] = {"tracker": tracker, "bbox": bbox}
        next_obj_id += 1
        track_ids.append(str(list(tracked_dict)[-1]))

for i, id in enumerate(track_ids):
    if id not in obj_tags:
        obj_tags[i] = id

